# Retaining Customers

Data reading and discovering

Data Exploration to choose the impact features 

Prediction Model

Model features

The steps are: 

Selecting the Customers 

In [1]:
import pandas as pd
df=pd.read_excel(r'')

In [63]:
#knowing the unique customers 
users = pd.DataFrame(df['customer_name'].unique())
users.columns = ['customer_name']
users.head()

how long the customers' accounts have been active

In [64]:
#last active transaction 
max_purchase_date = df.groupby('customer_name').purchase_date.max().reset_index()
max_purchase_date.columns = ['customer_name','max_purchase_date']
max_purchase_date.head()

In [65]:
# Compare the last transaction of the dataset with last transaction dates of the individual
max_purchase_date['last_active_time'] = (max_purchase_date['max_purchase_date'].max() - max_purchase_date['max_purchase_date']).dt.days
max_purchase_date.head()

In [66]:
#merge this dataframe to customers dataframe
users = pd.merge(users, max_purchase_date[['customer_name','last_active_time']], on='customer_name')
users.head()

In [67]:
users.boxplot('last_active_time')

In [20]:
from matplotlib import pyplot as plt

def cluster(x):    
    #clustering customers based on value
    from sklearn.cluster import KMeans

    sse={} # error
    df =x
    print(df)
    for k in range(1, 5): #you can change the range 
        kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df)
        df["clusters"] = kmeans.labels_  #cluster names 
        print(df["clusters"])
        sse[k] = kmeans.inertia_ #sse corresponding to clusters
    plt.figure()
    plt.plot(list(sse.keys()), list(sse.values()))
    plt.xlabel("Number of cluster")
    plt.show()

In [68]:
cluster(users[['last_active_time']])

In [69]:
from sklearn.cluster import KMeans
#building the clusters  and adding it to dataframe
kmeans = KMeans(n_clusters=4)  # you can change the number of clusters basd on your data 
users['last_active_time_Cluster'] = kmeans.fit_predict(users[['last_active_time']])
users.groupby('last_active_time_Cluster')['last_active_time'].describe()

number of times the customers use the services

In [70]:
frequency = df.groupby('customer_name').purchase_date.count().reset_index()
frequency.columns = ['customer_name','Frequency']
frequency

In [71]:
frequency['Frequency'].unique()

In [72]:
#you can classify it based on your data
#you have to change value based on your data
frequency['class']=1
#frequency.loc[frequency['Frequency']<value, 'class']=1
frequency.loc[(frequency['Frequency']<value) , 'class']=2
frequency.loc[frequency['Frequency']>=value, 'class']=3

In [73]:
users.columns

In [74]:
#merge this dataframe to custoners dataframe
users = pd.merge(users, frequency, on='customer_name')
users.head()

In [75]:
users.boxplot('Frequency')

In [76]:
users['Frequency_Cluster']=users['class']

In [77]:
cluster(users[['Frequency']])

In [78]:
#building the clusters for frequency
kmeans = KMeans(n_clusters=4) # you can change the number of clusters basd on your data 
users['Frequency_Cluster'] = kmeans.fit_predict(users[['Frequency']])
users

In [79]:
users.groupby('Frequency_Cluster')['Frequency'].describe()

total amount of paymnet for the customers 

In [80]:
total_paymnet = df.groupby('customer_name').billing.sum().reset_index()
total_paymnet

In [81]:
users = pd.merge(users, total_paymnet, on='customer_name')
users.head()

In [82]:
users.boxplot('billing')

In [83]:
cluster(users[['billing']])

In [84]:
#building the clusters 
kmeans = KMeans(n_clusters=3) # you can change the number of clusters basd on your data 
users['billing_Cluster'] = kmeans.fit_predict(users[['billing']])
users

In [85]:
users.groupby('billing_Cluster')['billing'].describe()

customers' account  age

In [86]:
#AccountLength  (how long account has been active)
account_age = df.groupby('customer_name').created_date.max().reset_index()
account_age.columns = ['customer_name','created_date']
account_age.head()

In [87]:
#computing the age 
account_age['account_lenght'] = (account_age['created_date'].max() - account_age['created_date']).dt.days
account_age.head()

In [88]:
#merge this dataframe to customers dataframe
users = pd.merge(users, account_age[['customer_name','account_lenght']], on='customer_name')
users.head()

In [89]:
users.boxplot('account_lenght')

In [90]:
cluster(users[['account_lenght']])

In [91]:
#building the clusters  and adding it to dataframe
kmeans = KMeans(n_clusters=4)  # you can change the number of clusters basd on your data 
users['account_lenght_Cluster'] = kmeans.fit_predict(users[['account_lenght']])
users.groupby('account_lenght_Cluster')['account_lenght'].describe()

Final Score

In [92]:
#final score
#calculate overall score and use mean() to see details
users['OverallScore'] = users['last_active_time_Cluster'] + users['Frequency_Cluster'] + users['billing_Cluster']+users['account_lenght_Cluster']
users.groupby('OverallScore')['last_active_time','account_lenght','Frequency','billing'].mean().reset_index()

In [93]:
users['OverallScore'].unique()

In [94]:
#segementation 
#you have to change value based on your data
users['Segment'] = 'Low'
# users.loc[users['OverallScore']> value,'Segment'] = 'Low' 
users.loc[(users['OverallScore']>=3 ) ,'Segment'] = 'Medium' 
users.loc[users['OverallScore']==8,'Segment'] = 'High' 

################################################################

Prediction

In [54]:
import pandas as pd 
import numpy as np
# import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_classif

In [56]:
df_1=users[['account_lenght','Frequency','billing', 'OverallScore']]

Features Importance

In [95]:
dfCorr = df_1.corr()
filteredDf = dfCorr[((dfCorr >= .7) & (dfCorr != 1)) ]
plt.figure(figsize=(30,10))
sns.heatmap(filteredDf, annot=True, cmap="Reds")
plt.show()

In [96]:
X = df_1.iloc[:,0:3]  #independent columns
y = df_1.iloc[:,-1]    #target column 
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_)
#plot graph of feature importances 
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(24).plot(kind='barh')
plt.figure(figsize=(80,90))
plt.show()

In [59]:
#splitting the data (to use it on the building the model)
y=df_1['OverallScore']
x=df_1.drop('OverallScore',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

Gradient Boosting Classifier

In [97]:
rf = GradientBoostingClassifier(n_estimators=100)
rf.fit(x_train, y_train)
train_pred = rf.predict(x_train)
print('train',train_pred)     
y_pred = rf.predict(x_test)
print('test',y_pred)

In [98]:
rf.fit(x_train,y_train)

In [99]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')